In [2]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import json
import cv2
import sklearn
from tensorflow.keras import regularizers
from tensorflow.keras import layers
import os

In [3]:
GESTURE_TYPES = 11
LABEL_DICT = {k:i for i,k in enumerate([21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])}
CONNECTION_LABELS = [
    (0, 1), (1, 2), (2, 3), (3, 4),
    (5, 6), (6, 7), (7, 8),
    (9, 10), (10, 11), (11, 12),
    (13, 14), (14, 15), (15, 16),
    (17, 18), (18, 19), (19, 20),
    (0, 5), (5, 9), (9, 13), (13, 17), (0, 17)
]

In [15]:
def load_keypoints(data_path='gesture_recognition/Fall 2020/data'):
    keypoints = []
    labels = []
    for subjectName in os.listdir(data_path):
        if not (subjectName.startswith("Subject") or subjectName.startswith("subject")): continue
        # subjectNum = int(re.findall(r'(\d+)', subjectName)[0])
        for sceneName in os.listdir(os.path.join(data_path, subjectName)):
            if not (sceneName.startswith("Scene") or subjectName.startswith("scene")): continue
            for groupEntry in os.scandir(os.path.join(data_path, subjectName, sceneName)):
                with open(groupEntry, 'r') as f:
                    groupData = json.load(f)
                    for gesture in groupData:
                        for i in range(len(gesture['keypoints'])):
                            if not gesture['keypoints'][i]:
                                continue
                            keypoints.append(gesture['keypoints'][i])
                            labels.append(LABEL_DICT[gesture['label']])
    keypoints = np.array(keypoints)
    labels = np.array(labels)
    return keypoints, labels
keypoints, labels = load_keypoints()
print(keypoints.shape, labels.shape)

(33448, 21, 3) (33448,)


In [4]:
def normalize(keypoints):
    for i in range(len(keypoints)):
        keypoints[i] = 1 - keypoints[i]
        keypoints[i] -= keypoints[i, 0]
        keypoints[i] /= np.linalg.norm(keypoints[i, 0] - keypoints[i, 9])
        theta = -np.arcsin(keypoints[i, 9, 0])
        rotation = np.array([
            [np.cos(theta), -np.sin(theta)],
            [np.sin(theta), np.cos(theta)]
        ])
        keypoints[i, :, :2] = keypoints[i, :, :2] @ rotation

        theta2 = np.arcsin(keypoints[i, 9, 2])
        rotation2 = np.array([
            [np.cos(theta2), -np.sin(theta2)],
            [np.sin(theta2), np.cos(theta2)]
        ])
        keypoints[i, :, 1:3] = keypoints[i, :, 1:3] @ rotation2
# normalize(keypoints)

# print(keypoints.shape, labels.shape)

(15442, 21, 3) (15442,)


In [5]:
def generate_connection_angles(keypoints, keypoints_num=21, keypoints_dimensions=3):
    connections = []
    for connection in CONNECTION_LABELS:
        connections.append(keypoints[..., connection[1], :] - keypoints[..., connection[0], :])
    connections = np.stack(connections, axis = -2)
    tensor1 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,3,2)
    tensor2 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,3,1,2)
    angles = (tensor1*tensor2).sum(axis=-1)/np.linalg.norm(tensor1,axis=-1)/np.linalg.norm(tensor2,axis=-1)
    angles = angles.transpose(1,2,0)[np.triu_indices(21, k = 1)].transpose(1,0)
    return np.arccos(angles)
angles = generate_connection_angles(keypoints)
print(angles.shape)

(15442, 210)


In [21]:
def visualize_keypoints(keypoints):
    for points in keypoints:
        img = np.zeros((480, 640, 3))
        for point in points:
            x, y, z = point
            if not x:
                continue
            cv2.circle(img, (int(x), int(y)), 4, (255, 0, 0), 2)
        for connection in CONNECTION_LABELS:
            if not points[connection[0]][0]:
                continue
            x0, y0, z0 = points[connection[0]]
            x1, y1, z1 = points[connection[1]]
            cv2.line(img, (int(x0), int(y0)), (int(x1), int(y1)), (0, 255, 0), 2)
        cv2.imshow("Key Points", img)
        key = cv2.waitKey(1)
        if key == 27:
            cv2.destroyAllWindows()
            cv2.waitKey(1) # cannot close window on macOS without this line
            return
# visualize_keypoints(keypoints)

In [7]:
def process_features(keypoints, angles):
    data_length = keypoints.shape[0]
    keypoints = keypoints.reshape(data_length, -1)
    angles = angles.reshape(data_length, -1)
    features = np.concatenate((keypoints, angles), -1)
    return features
X = process_features(keypoints, angles)
#normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
#normalizer.adapt(X)
# X_train, X_val, y_train, y_val = train_test_split(processed_keypoints, labels, test_size=0.2, random_state=0)
print(X.shape)

(15442, 273)


In [8]:
model_mlp = tf.keras.Sequential([
    tf.keras.layers.Dense(GESTURE_TYPES, activation=None), 
    tf.keras.layers.Activation('softmax')])
model_mlp.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_mlp.fit(X, labels, epochs=50, validation_split=0.2)

Epoch 1/50
387/387 [==============================] - 1s 2ms/step - loss: 1.3642 - accuracy: 0.6084 - val_loss: 0.6313 - val_accuracy: 0.8084
Epoch 2/50
387/387 [==============================] - 0s 1ms/step - loss: 0.5995 - accuracy: 0.8380 - val_loss: 0.5404 - val_accuracy: 0.8333
Epoch 3/50
387/387 [==============================] - 0s 1ms/step - loss: 0.5162 - accuracy: 0.8545 - val_loss: 0.5008 - val_accuracy: 0.8478
Epoch 4/50
387/387 [==============================] - 0s 1ms/step - loss: 0.4643 - accuracy: 0.8724 - val_loss: 0.5169 - val_accuracy: 0.8410
Epoch 5/50
387/387 [==============================] - 0s 1ms/step - loss: 0.4437 - accuracy: 0.8676 - val_loss: 0.4932 - val_accuracy: 0.8540
Epoch 6/50
387/387 [==============================] - 0s 1ms/step - loss: 0.4204 - accuracy: 0.8795 - val_loss: 0.4948 - val_accuracy: 0.8498
Epoch 7/50
387/387 [==============================] - 0s 1ms/step - loss: 0.4015 - accuracy: 0.8808 - val_loss: 0.4623 - val_accuracy: 0.8650
Epoch 

In [24]:
model_mlp = tf.keras.Sequential([
    tf.keras.layers.Dense(GESTURE_TYPES, activation=None), 
    tf.keras.layers.Dense(256, activation=None), 
    tf.keras.layers.Activation('softmax')])
model_mlp.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_mlp.fit(X, labels, epochs=30, validation_split=0.2)

Epoch 1/30
387/387 [==============================] - 1s 3ms/step - loss: 2.8241 - accuracy: 0.5274 - val_loss: 0.6726 - val_accuracy: 0.8436
Epoch 2/30
387/387 [==============================] - 1s 3ms/step - loss: 0.5959 - accuracy: 0.8635 - val_loss: 0.5660 - val_accuracy: 0.8634
Epoch 3/30
387/387 [==============================] - 1s 3ms/step - loss: 0.4589 - accuracy: 0.8861 - val_loss: 0.5121 - val_accuracy: 0.8760
Epoch 4/30
387/387 [==============================] - 1s 3ms/step - loss: 0.4150 - accuracy: 0.8896 - val_loss: 0.4995 - val_accuracy: 0.8660
Epoch 5/30
387/387 [==============================] - 1s 3ms/step - loss: 0.3752 - accuracy: 0.8985 - val_loss: 0.4697 - val_accuracy: 0.8725
Epoch 6/30
387/387 [==============================] - 2s 4ms/step - loss: 0.3476 - accuracy: 0.9025 - val_loss: 0.4488 - val_accuracy: 0.8737
Epoch 7/30
387/387 [==============================] - 2s 4ms/step - loss: 0.3536 - accuracy: 0.8997 - val_loss: 0.4567 - val_accuracy: 0.8737
Epoch 

In [25]:
model_lstm_mtm = tf.keras.Sequential([layers.Masking(), layers.LSTM(GESTURE_TYPES, activation=None, return_sequences=True), layers.Activation('softmax')])
model_lstm_mtm.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_lstm_mtm.fit(X, labels, epochs=20, validation_split=0.2)

Epoch 1/20


ValueError: in user code:

    C:\Users\Michael\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\Michael\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:843 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Michael\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:1280 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Michael\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:2820 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Michael\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:3588 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\U